The goal of this notebook is to understand the original data structure and define basic extraction operations.

The source data explored here has been previously downloaded from Path of Exile stash tabs API to local .txt files. Each file contained in a specific folder represents a single API call to http://www.pathofexile.com/api/public-stash-tabs?id= with a custom **nextChangeId**: this id is iteratively applyed on the next API call and can be found inside the content of the previous loaded data.

# TODO

In [1]:
import os
import re
import json
import random

import pandas as pd

In [7]:
pd.set_option('display.max_columns', 500)

DATA_PATH = '/home/fabio/Desktop/poe_price/data/temp/stashes_harvest/'

LEAGUE = 'Harvest'

CURRENCY_KEY = ['alt', 'fuse', 'alch', 'chaos', 'gcp', 'exa', 'chrom', 'jew', 'chance', 'chisel', 'scour', 'blessed', 'regret', 'regal', 'divine', 'vaal', 'silver']
CURRENCY_NAME = ["Orb of Alteration", "Orb of Fusing", "Orb of Alchemy", "Chaos Orb", "Gemcutter's Prism", "Exalted Orb", "Chromatic Orb", "Jeweller's Orb", "Orb of Chance", "Cartographer's Chisel", "Orb of Scouring", "Blessed Orb", "Orb of Regret", "Regal Orb", "Divine Orb", "Vaal Orb", "Silver Coin"]
FRAME_TYPES = {0: 'normal', 1: 'magic', 2:'rare', 3: 'unique', 4: 'gem', 5: 'currency', 6: 'divination card', 7: 'quest item', 8: 'prophecy', 9: 'relic'}


ITEMS_CATEGORIES = ['accessories', 'armour', 'jewels', 'weapons', 'currency']



In [73]:
def get_stashes_dict(path):
    with open(path, 'rb') as file:
        try:
            return json.load(file)['stashes']
        except:
            print('ERROR: ', path)
            return []
        
def get_stashes_items(verbose=False, target_category=None):
    items = list()
    N = 100
    files = os.listdir(DATA_PATH)
    
    # select N random values if possible
    files = random.sample(files, N if len(files) > N else len(files))       
        
    # select a fixed number of files
    for v in files:
        # for v in os.listdir(DATA_PATH):
        for stash in get_stashes_dict(DATA_PATH + v):
            if len(stash['items']) > 0 and stash['public']:
                # add stash related data to single items
                for item in stash['items']:
                    item['stashNote'] = stash['stash']
                    item['league'] = stash['league']
                    
                    if target_category is not None and item['extended']['category'] in target_category:
                        items.append(item)
                    elif target_category is None:
                        items.append(item)
                    else:
                        pass
    return items

def get_string_price(string):
    if re.match('(~price|~b/o)\s\d+(\.\d+)?(/\d+(\.\d+)?)?\s\w+', string) is not None:
        tokens = string.split(' ')

        currency, quantity = tokens[2], tokens[1]
        if re.match('\d+((\.|/)\d+)?/\d+((\.|/)\d+)?', quantity):
            quantity, sellingStackSize = quantity.split('/')
            quantity = round(float(quantity), 2)
            sellingStackSize = round(float(sellingStackSize), 2)
        else:
            quantity = round(float(quantity), 2)
            sellingStackSize = None

        # check if currency and quantity are allowed values
        if currency in CURRENCY_KEY:
            return currency, quantity, 1 if sellingStackSize is None else sellingStackSize
    raise ValueError()
    
def get_empty_columns(df):
    return [col for col in df.columns if all(df[col].isna())]

# Data visualization

In [71]:
items = pd.DataFrame(get_stashes_items())

# alphabetically order dataframe columns
items = items[sorted(items.columns)]

items.shape

(224052, 54)

In [10]:
items.sample(10)

,abyssJewel,additionalProperties,artFilename,corrupted,cosmeticMods,craftedMods,delve,descrText,duplicated,elder,enchantMods,explicitMods,extended,flavourText,fractured,fracturedMods,frameType,h,harvestRequirements,hybrid,icon,id,identified,ilvl,implicitMods,incubatedItem,influences,inventoryId,itemLevel,league,maxStackSize,name,nextLevelRequirements,note,properties,prophecyText,requirements,secDescrText,shaper,socketedItems,sockets,stackSize,stashNote,support,synthesised,talismanTier,typeLine,utilityMods,veiled,veiledMods,verified,w,x,y
10110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[+18 to all Attributes, 124% increased Armour ...","{'category': 'armour', 'subcategories': ['helm...","[""To touch it is to be overwhelmed with the ne...",NaN,NaN,3,2,NaN,NaN,https://web.poecdn.com/image/Art/2DItems/Armou...,564a65a25b447aa00217f5f8b55b1ebb28c4def5a91574...,True,78,NaN,NaN,NaN,Stash1,NaN,Harvest,NaN,Ahn's Contempt,NaN,NaN,"[{'name': 'Armour', 'values': [['361', 1]], 'd...",NaN,"[{'name': 'Level', 'values': [['68', 0]], 'dis...",NaN,NaN,[],"[{'group': 0, 'attr': 'I', 'sColour': 'B'}]",NaN,U,NaN,NaN,NaN,Praetor Crown,NaN,NaN,NaN,False,2,92,0
45896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A stack of 20 shards becomes an Orb of Alterat...,NaN,NaN,NaN,NaN,"{'category': 'currency', 'baseType': 'Alterati...",NaN,NaN,NaN,5,1,NaN,NaN,https://web.poecdn.com/image/Art/2DItems/Curre...,4129987fdf55beea8649f77cfa0c77e0e535070ee3efc5...,True,0,NaN,NaN,NaN,Stash12,NaN,Harvest,20.0,,NaN,NaN,"[{'name': 'Stack Size', 'values': [['14/20', 0...",NaN,NaN,NaN,NaN,NaN,NaN,14.0,Trade,NaN,NaN,NaN,Alteration Shard,NaN,NaN,NaN,False,1,0,8
131148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Right click to drink. Can only hold charges wh...,NaN,NaN,NaN,"[35% increased Charge Recovery, Immunity to Bl...","{'category': 'flasks', 'prefixes': 1, 'suffixe...",NaN,NaN,NaN,1,2,NaN,NaN,https://web.poecdn.com/gen/image/WzksMTQseyJmI...,9f6ee0fdba5ebb9e4d22298502dd8b9065f80a3580f9b6...,True,70,NaN,NaN,NaN,Stash10,NaN,Harvest,NaN,,NaN,NaN,"[{'name': 'Lasts %0 Seconds', 'values': [['4.0...",NaN,"[{'name': 'Level', 'values': [['6', 0]], 'disp...",NaN,NaN,NaN,NaN,NaN,Flasks,NaN,NaN,NaN,Perpetual Quicksilver Flask of Staunching,[40% increased Movement Speed],NaN,NaN,False,1,0,6
9362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Regenerate 10.6 Life per second, +37 to maxim...","{'category': 'accessories', 'subcategories': [...",NaN,NaN,NaN,2,1,NaN,NaN,https://web.poecdn.com/image/Art/2DItems/Amule...,99db1414384a4066aed0242f10b69e8c316350bf2f1358...,True,84,[+21 to Dexterity and Intelligence],NaN,NaN,Stash38,NaN,Harvest,NaN,Hate Rosary,NaN,NaN,NaN,NaN,"[{'name': 'Level', 'values': [['47', 0]], 'dis...",NaN,NaN,NaN,NaN,NaN,~price 45 chaos,NaN,NaN,NaN,Turquoise Amulet,NaN,NaN,NaN,False,1,6,11
115538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[+32% to Lightning Resistance, Minions have 30...","{'category': 'weapons', 'subcategories': ['wan...",NaN,NaN,NaN,2,3,NaN,NaN,https://web.poecdn.com/image/Art/2DItems/Weapo...,6d72ce6db82b0f95857b61c590fd0b6f82f064e6c0e3a5...,True,83,[Can roll Minion Modifiers],NaN,NaN,Stash48,NaN,Harvest,NaN,Chimeric Barb,NaN,~price 1 exalted,"[{'name': 'Wand', 'values': [], 'displayMode':...",NaN,"[{'name': 'Level', 'values': [['72', 0]], 'dis...",NaN,NaN,[],"[{'group': 0, 'attr': 'D', 'sColour': 'G'}, {'...",NaN,Sell,NaN,NaN,NaN,Convoking Wand,NaN,NaN,NaN,False,1,4,20
208257,NaN,"[{'name': 'Experience', 'values': [['1664/2604...",NaN,NaN,NaN,NaN,NaN,This is a Support Gem. It does not grant a bon...,NaN,NaN,NaN,[Gain Arcane Surge after Spending a total of 2...,"{'category': 'gems', 'subcategories': ['suppor...",NaN,NaN,NaN,4,1,NaN,NaN,https://web.poecdn.com/image/Art/2DItems/Gems/...,83b84be2235b3191e92732840ab7ec9b0bffcd61e67b9f...,True,0,NaN,NaN,NaN,Stash20,NaN,Harvest,NaN,,NaN,NaN,"[{'name': 'Support, Spell, Duration', 'values'...",NaN,"[{'name': 'Level', 'values': [['11', 0]], 'dis...",Each supported spell will track how much mana ...,NaN,NaN,NaN,NaN,3,True,NaN,NaN,Arcan

# Data processing

## Initial manipulation and filtering

In [11]:
# extract items category and subcategory
items['subCategory'] = items.extended.apply(lambda y: ' '.join(y['subcategories']) 
                                            if 'subcategories' in y else None)
items['category'] = items.extended.apply(lambda y: y['category'])

# drop unwanted categories
items = items[items.category.isin(ITEMS_CATEGORIES)]

# filter out unwanted subcategories
items.drop(index=items[(items.subCategory=='cluster') & (items.category=='jewels') ].index, 
           inplace=True)

items.shape

(146594, 56)

In [74]:
# extract items price and drop unpriced items
price = []
for k, v in items.iterrows():
    try:
        # item has a valid price
        item_price = get_string_price(v.note)
    except:
        try:
            # stash has a valid price
            item_price = get_string_price(v.stashNote)
        except:
            # both item and stash have no valid price
            item_price = (None, None, None)
    price.append(item_price)
(items['priceCurrency'], items['priceQuantity'], items['sellingQuantity']) = zip(*price)

# filter out unwanted selling currencies
items = items[(items.priceCurrency.isin(CURRENCY_KEY)) & (items.priceQuantity != 0)]

items.shape

'chaos', 35.0, 1)
~price 35 chaos ('chaos', 35.0, 1)
~price 6 chaos ('chaos', 6.0, 1)
~price 5 chaos ('chaos', 5.0, 1)
~price 7 chaos ('chaos', 7.0, 1)
~price 50 chaos ('chaos', 50.0, 1)
~price 80 chaos ('chaos', 80.0, 1)
~price 4 chaos ('chaos', 4.0, 1)
~b/o 2.5 chaos ('chaos', 2.5, 1)
~price 1 chaos ('chaos', 1.0, 1)
~price 70 chaos ('chaos', 70.0, 1)
~price 5 chaos ('chaos', 5.0, 1)
~price 20 chaos ('chaos', 20.0, 1)
~price 100 chaos ('chaos', 100.0, 1)
~price 12 chaos ('chaos', 12.0, 1)
~price 10 chaos ('chaos', 10.0, 1)
~price 14 chaos ('chaos', 14.0, 1)
~price 10 chaos ('chaos', 10.0, 1)
~price 10 chaos ('chaos', 10.0, 1)
~price 15 chaos ('chaos', 15.0, 1)
~price 15 chaos ('chaos', 15.0, 1)
~price 70 chaos ('chaos', 70.0, 1)
~price 10 chaos ('chaos', 10.0, 1)
~price 50 chaos ('chaos', 50.0, 1)
~price 10 chaos ('chaos', 10.0, 1)
~price 20 chaos ('chaos', 20.0, 1)
~price 20 chaos ('chaos', 20.0, 1)
~price 10 chaos ('chaos', 10.0, 1)
~price 10 chaos ('chaos', 10.0, 1)
~price 10 chao

(47873, 57)

## Mod based items processing

In [13]:
mitems = items[items.category.isin(['jewels', 'armour', 'weapons', 'accessories'])].copy()
mitems.shape

(35261, 59)

In [14]:
mitems['rarity'] = mitems.frameType.apply(lambda y: FRAME_TYPES[y])
mitems = mitems[(mitems.rarity.isin(['normal', 'magic', 'rare'])) | (mitems.category=='currency')]
mitems.shape

(28030, 60)

In [15]:
mitems['nPrefixes'] = mitems.extended.apply(lambda y: y['prefixes'] if 'prefixes' in y else None)
mitems['nSuffixes'] = mitems.extended.apply(lambda y: y['suffixes'] if 'suffixes' in y else None)

In [16]:
# explode "influences" column into distinct dataframe columns
mitems_influences = mitems['influences'].apply(lambda y: y if isinstance(y, dict) else {})
mitems_influences = pd.DataFrame(mitems_influences.to_list(), index=mitems_influences.index)
mitems_influences.columns = ['influence{}'.format(v.capitalize()) for v in mitems_influences.columns]
mitems = pd.merge(mitems, mitems_influences, left_index=True, right_index=True, how='left')
mitems.shape

(28030, 68)

In [17]:
def mitems_prop_formatting(mitems, target_props):
    mitems_props_vocabulary = []
    mitems_props = []
    
    t1 = []
    for k, v in mitems.iterrows():
        for target_prop in target_props:
            if isinstance(v[target_prop], list):
                for prop in [p for p in v[target_prop] if len(p['values']) > 0]:
                    item_prop = dict()
                    item_prop['itemId'] = k
                    generic_prop = prop['name']
                    try:
                        prop_index = mitems_props_vocabulary.index(generic_prop)
                    except:
                        mitems_props_vocabulary.append(generic_prop)
                        prop_index = len(mitems_props_vocabulary) - 1
                    item_prop['propId'] = prop_index
                    for i, v in enumerate(prop['values'][0][0].split('-')):
                        try:
                            item_prop['value{}'.format(i)] = float(re.sub('(\+|-|%)', '', v))
                        except:
                            break
                    mitems_props.append(item_prop)
    mitems_props = pd.DataFrame(mitems_props)
    mitems_props_vocabulary = pd.DataFrame(mitems_props_vocabulary, columns=['text'])
    return mitems_props, mitems_props_vocabulary

def mitems_mod_formatting(mitems, mod_types):
    mitems_mods_vocabulary = []
    mitems_mods = []
    for k, item in mitems.iterrows():
        for mod_type in mod_types:
            if isinstance(item[mod_type], list):
                for mod in item[mod_type]:
                    item_mod = dict()
                    item_mod['itemId'] = k
                    generic_mod = re.sub('\d+', '#', mod)
                    try:
                        mod_index = mitems_mods_vocabulary.index((generic_mod, mod_type))
                    except:
                        mitems_mods_vocabulary.append((generic_mod, mod_type))
                        mod_index = len(mitems_mods_vocabulary) - 1
                    item_mod['modId'] = mod_index
                    for i, v in enumerate(re.findall('\d+', mod)):
                        item_mod['value{}'.format(i)] = int(v)
                    mitems_mods.append(item_mod)
    mitems_mods = pd.DataFrame(mitems_mods)
    mitems_mods_vocabulary = pd.DataFrame(mitems_mods_vocabulary, columns=['text', 'modType'])
    return mitems_mods, mitems_mods_vocabulary

mitems_prop, mitems_prop_voc = mitems_prop_formatting(mitems[mitems.category!='currency'], ['properties'])
mitems_mods, mitems_mods_voc = mitems_mod_formatting(mitems[mitems.category!='currency'], ['craftedMods', 'enchantMods', 'explicitMods', 'implicitMods'])

# jewels only
# mitems_mods, mitems_mods_voc = mitems_mod_formatting(mitems[mitems.category!='currency'], ['explicitMods', 'implicitMods'])

In [18]:
mitems_req = mitems['requirements'].apply(lambda y: {req['name']: int(req['values'][0][0]) for req in y} if isinstance(y, list) else {})
mitems_req = pd.DataFrame(mitems_req.to_list(), index=mitems_req.index)
mitems_req.columns = ['requirement{}'.format(col_name.capitalize()) for col_name in mitems_req.columns]

if 'requirementStrength' in mitems_req:
    mitems_req.loc[mitems_req.requirementStrength.notna(), 'requirementStr'] = mitems_req[mitems_req.requirementStrength.notna()].requirementStrength.values

if 'requirementDexterity' in mitems_req:
    mitems_req.loc[mitems_req.requirementDexterity.notna(), 'requirementDex'] = mitems_req[mitems_req.requirementDexterity.notna()].requirementDexterity.values

if 'requirementIntelligence' in mitems_req:    
    mitems_req.loc[mitems_req.requirementIntelligence.notna(), 'requirementInt'] = mitems_req[mitems_req.requirementIntelligence.notna()].requirementIntelligence.values

mitems_req.drop(columns=['requirementStrength', 'requirementDexterity', 'requirementIntelligence'], inplace=True, errors='ignore')

mitems = pd.merge(mitems, mitems_req, left_index=True, right_index=True, how='left')

mitems.shape

(28030, 72)

In [19]:
# split mitems socket column into separate dataframe
for k, v in mitems.iterrows():
    item_sockets = v['sockets']
    if isinstance(item_sockets, list):
        for socket in item_sockets:
            socket['itemId'] = k
mitems_sockets = pd.DataFrame([socket for sockets in mitems[mitems.sockets.notna()].sockets.to_list() for socket in sockets])
mitems_sockets.drop(columns=['attr'], inplace=True)

In [20]:
mitems['nVeiledMods'] = mitems.veiledMods.apply(lambda y: len(y) if isinstance(y, list) else 0)

In [21]:
# useless variables
mitems.drop(columns=[
    'abyssJewel', 'additionalProperties', 'artFilename', 'cosmeticMods', 'craftedMods', 
    'delve', 'descrText', 'elder', 'enchantMods', 'explicitMods', 'extended', 'flavourText', 'fractured', 
    'fracturedMods', 'frameType', 'h', 'harvestRequirements', 'hybrid', 'icon', 'id', 'implicitMods', 'incubatedItem', 'influences',
    'inventoryId', 'isRelic', 'maxStackSize', 'name', 'nextLevelRequirements',
    'note', 'properties', 'prophecyText', 'requirements', 'secDescrText', 'shaper',
    'socketedItems', 'sockets', 'stackSize', 'stashNote', 'support', 'typeLine', 'utilityMods',
    'veiledMods', 'veiled', 'verified', 'w', 'x', 'y', 'itemLevel', 'sellingStackSize'
    ], inplace=True, errors='ignore')

In [22]:
mitems = mitems[sorted(mitems.columns)]
mitems.sample(5)

,category,corrupted,duplicated,harvestRequirements,identified,ilvl,influenceCrusader,influenceElder,influenceHunter,influenceRedeemer,influenceShaper,influenceWarlord,league,nPrefixes,nSuffixes,nVeiledMods,priceCurrency,priceQuantity,rarity,requirementDex,requirementInt,requirementLevel,requirementStr,sellingQuantity,subCategory,synthesised,talismanTier
191570,accessories,NaN,NaN,NaN,True,73,NaN,NaN,NaN,NaN,NaN,NaN,Harvest,2.0,3.0,0,chaos,22,rare,NaN,NaN,64.0,NaN,None,ring,NaN,NaN
122490,armour,NaN,NaN,NaN,True,83,NaN,NaN,NaN,NaN,True,NaN,Harvest,3.0,1.0,0,chaos,14,rare,136.0,NaN,64.0,NaN,None,shield,NaN,NaN
136472,accessories,NaN,NaN,NaN,True,72,NaN,NaN,NaN,NaN,NaN,NaN,Harvest,2.0,2.0,0,chaos,8,rare,NaN,NaN,57.0,NaN,None,ring,NaN,NaN
569,accessories,NaN,NaN,NaN,True,80,NaN,NaN,NaN,NaN,NaN,NaN,Harvest,3.0,3.0,0,chaos,10,rare,NaN,NaN,48.0,NaN,None,belt,NaN,NaN
35005,accessories,NaN,NaN,NaN,True,83,True,NaN,NaN,NaN,NaN,NaN,Harvest,2.0,3.0,0,chaos,10,rare,NaN,NaN,64.0,NaN,None,ring,NaN,NaN


In [23]:
mitems_sockets.sample(5)

,group,sColour,itemId
15909,0,B,109754
6959,1,G,49247
30202,0,A,202856
6326,0,G,44669
31312,0,R,210053


In [24]:
mitems_mods_voc.sample(5)

,text,modType
1013,Socketed Gems are supported by Level # Additio...,explicitMods
279,+# Mana gained when you Block,explicitMods
1381,#% increased Ethereal Knives Projectile Speed,enchantMods
506,#% chance to Avoid Cold Damage from Hits,explicitMods
1015,Warcry Skills have #% increased Area of Effect,explicitMods


In [25]:
mitems_mods.sample(5)

,itemId,modId,value0,value1,value2
25520,42893,6,6.0,NaN,NaN
19832,30819,19,60.0,NaN,NaN
58986,94281,30,32.0,NaN,NaN
124235,197277,29,11.0,NaN,NaN
112236,178911,29,16.0,NaN,NaN


In [26]:
display(mitems_prop.sample(5))
display(mitems_prop_voc.sample(5))

,itemId,propId,value0,value1
9607,79191,0,261.0,NaN
1250,9053,2,94.0,175.0
8941,73407,4,6.0,NaN
17042,133797,9,24.0,NaN
17758,141127,0,114.0,NaN


,text
0,Evasion Rating
11,Quality (Attack Modifiers)
5,Attacks per Second
8,Quality
6,Weapon Range


### Jewels

In [29]:
jewel = mitems[mitems.category=='jewels'].copy()
jewel.shape

(7470, 27)

In [30]:
jewel['isAbyss'] = jewel.subCategory.apply(lambda y: y == 'abyss')

In [31]:
jewel.drop(columns=['category', 'influenceCrusader', 'influenceElder', 'influenceHunter', 'influenceRedeemer', 
                    'influenceShaper', 'influenceWarlord', 'itemLevel', 'nVeiledMods', 'requirementDex', 
                    'requirementInt', 'requirementStr', 'subCategory', 'synthesised',
                    'talismanTier', 'veiled'],
          inplace=True, errors='ignore')

In [32]:
jewel.sample()

,corrupted,duplicated,harvestRequirements,identified,ilvl,league,nPrefixes,nSuffixes,priceCurrency,priceQuantity,rarity,requirementLevel,sellingQuantity,isAbyss
202866,NaN,NaN,NaN,True,82,Harvest,1.0,2.0,chaos,5,rare,NaN,None,False


### Accessories

In [118]:
accessory = mitems[mitems.category=='accessories'].copy()
accessory.shape

(9689, 26)

In [119]:
accessory.drop(columns=['category', 'itemLevel', 'requirementDex', 'requirementInt', 
                        'requirementStr'],
              inplace=True, errors='ignore')

In [120]:
accessory.columns

Index(['corrupted', 'duplicated', 'identified', 'ilvl', 'influenceCrusader',
       'influenceElder', 'influenceHunter', 'influenceRedeemer',
       'influenceShaper', 'influenceWarlord', 'league', 'nPrefixes',
       'nSuffixes', 'nVeiledMods', 'priceCurrency', 'priceQuantity', 'rarity',
       'requirementLevel', 'subCategory', 'synthesised', 'talismanTier',
       'veiled'],
      dtype='object')

### Weapons

In [144]:
weapon = mitems[mitems.category=='weapons'].copy()
weapon.shape

(3725, 25)

In [145]:
weapon.drop(columns=['talismanTier'],
              inplace=True, errors='ignore')

In [146]:
weapon.columns

Index(['category', 'corrupted', 'duplicated', 'identified', 'ilvl',
       'influenceCrusader', 'influenceElder', 'influenceHunter',
       'influenceRedeemer', 'influenceShaper', 'influenceWarlord', 'league',
       'nPrefixes', 'nSuffixes', 'nVeiledMods', 'priceCurrency',
       'priceQuantity', 'rarity', 'requirementDex', 'requirementInt',
       'requirementLevel', 'requirementStr', 'subCategory', 'synthesised',
       'veiled'],
      dtype='object')

### Armour

In [163]:
armour = mitems[mitems.category=='armour'].copy()
armour.shape

(9548, 25)

In [164]:
armour.drop(columns=['talismanTier'],
              inplace=True, errors='ignore')

In [165]:
armour.columns

Index(['category', 'corrupted', 'duplicated', 'identified', 'ilvl',
       'influenceCrusader', 'influenceElder', 'influenceHunter',
       'influenceRedeemer', 'influenceShaper', 'influenceWarlord', 'league',
       'nPrefixes', 'nSuffixes', 'nVeiledMods', 'priceCurrency',
       'priceQuantity', 'rarity', 'requirementDex', 'requirementInt',
       'requirementLevel', 'requirementStr', 'subCategory', 'synthesised',
       'veiled'],
      dtype='object')

## Currencies

In [67]:
currency = items[items.category=='currency'].copy()
currency.shape

(2691, 59)

In [68]:
currency = currency[currency.extended.apply(lambda y: y['baseType'] in CURRENCY_NAME)]
currency['sell_currency'] = currency.extended.apply(lambda y: CURRENCY_KEY[CURRENCY_NAME.index(y['baseType'])])
currency.shape

(359, 60)

In [61]:
print(sum(currency.sellingQuantity.isna()))
currency.sellingQuantity.fillna(1, inplace=True)

85


In [69]:
currency.drop(columns=['abyssJewel', 'additionalProperties', 'artFilename', 'corrupted', 'cosmeticMods', 
    'craftedMods', 'delve', 'descrText', 'duplicated', 'elder', 'enchantMods', 'explicitMods',
     'flavourText', 'fractured', 'fracturedMods', 'frameType', 'h', 'harvestRequirements', 'hybrid', 
    'icon', 'id', 'identified', 'ilvl', 'implicitMods', 'incubatedItem', 'influences',
    'inventoryId', 'isRelic', 'itemLevel', 'name', 'nextLevelRequirements', 'note', 
    'properties', 'prophecyText', 'requirements', 'secDescrText', 'shaper', 'socketedItems', 'sockets',
    'stashNote', 'support', 'synthesised', 'talismanTier', 'typeLine', 'utilityMods', 'veiled', 
    'verified', 'veiledMods',  'w', 'x', 'y', 'subCategory', 'category', 'maxStackSize', 'stackSize'],
    inplace=True, errors='ignore')

In [63]:
currency.sample()[['extended']].values

array([[{'category': 'currency', 'baseType': 'Orb of Alchemy'}]],
      dtype=object)

In [70]:
currency[currency.sellingQuantity.isna()]

,extended,league,priceCurrency,priceQuantity,sellingQuantity,sell_currency
2151,"{'category': 'currency', 'baseType': 'Orb of A...",Harvest,chaos,60,None,alt
2157,"{'category': 'currency', 'baseType': 'Orb of A...",Harvest,chaos,20,None,alt
3007,"{'category': 'currency', 'baseType': 'Exalted ...",Harvest,chaos,200,None,exa
13521,"{'category': 'currency', 'baseType': 'Exalted ...",Harvest,chaos,155,None,exa
13970,"{'category': 'currency', 'baseType': 'Regal Orb'}",Harvest,chaos,1,None,regal
...,...,...,...,...,...,...
184598,"{'category': 'currency', 'baseType': 'Orb of A...",Harvest,chaos,10,None,alch
207620,"{'category': 'currency', 'baseType': 'Exalted ...",Harvest,chaos,150,None,exa
209329,"{'category': 'currency', 'baseType': 'Orb of C...",Harvest,chaos,25,None,chance
209585,"{'category': 'currency', 'baseType': 'Exalted ...",Harvest,alt,1200,None,exa
